# Reinforcement Learning GAN
Employ REINFORCE for training the Generator in a GAN formuluation.  As before, the reward to the Generator $G$ is simply the probability of real at each time point, $p_t$, assigned by the Discriminator $D$.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.distributions import Categorical

rnn_gen_size = 3
rnn_dis_size = 2
time_steps = 4
batch_size = 1
num_trials = 1000
print_every = 100
lr = 0.1
decay = 0.9

## Real Data.
Create a simple data distribution with sequential structure for testing the GAN.

In [2]:
num_examples = 1000

real_seq = np.array([[1., 0., 0.],[0., 1., 0.],[0., 0., 1.], [1., 0., 0.]])

# assert 
# real_data = np.
print real_seq

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]


## Generator and Discriminator.

In [3]:
def generator():
    '''Define the Generator graph.'''
    # TODO: Generalize for random input.
    # batch_indices = gen_z
    batch_indices = tf.constant([[1., 0., 0.]], dtype=tf.float32)   
    
    with tf.variable_scope('gen'):
        cell_gen = tf.nn.rnn_cell.BasicRNNCell(rnn_gen_size)
        state_gen = cell_gen.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnn'):
            sequence = []

            for t in xrange(time_steps):        
                if t > 0:
                    tf.get_variable_scope().reuse_variables()
                    
                rnn_in = batch_indices
                rnn_out, state_gen = cell_gen(rnn_in, state_gen)
                log_probs = tf.nn.log_softmax(rnn_out)

                index = tf.contrib.bayesflow.stochastic_graph.DistributionTensor(
                    tf.contrib.distributions.Categorical,
                    logits=log_probs)
                batch_indices = tf.one_hot(index, rnn_gen_size, dtype=tf.float32)
                sequence.append(batch_indices)
    return sequence


def discriminator(sequence):
    '''Define the Discriminator graph.'''
    with tf.variable_scope('dis'):    
        cell_dis = tf.nn.rnn_cell.BasicRNNCell(rnn_dis_size)
        state_dis = cell_dis.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnn'):
            predictions = []

            for t, inp in enumerate(sequence):
                rnn_in = inp
                if t > 0:
                    tf.get_variable_scope().reuse_variables()

                rnn_out, state_dis = cell_dis(rnn_in, state_dis)
                pred = tf.nn.sigmoid(rnn_out)
                predictions.append(pred)
    return predictions

## Training Operations.

In [31]:
def train_generator(predictions):
    '''Train generator via REINFORCE.'''
    rewards = []
    for pred in predictions:
        rewards.append(pred)
    
    # Exponential baseline.
    ema = tf.train.ExponentialMovingAverage(decay = decay)
    rewards_tf = tf.pack(rewards)
    reduced_reward = tf.reduce_mean(rewards_tf)
    maintain_avg_op = ema.apply([reduced_reward])
    baseline = ema.average(reduced_reward)
    
    # Advantage.
    loss = []
    for reward in rewards:
        advantage = reward - baseline
        loss.append(-advantage)

    # Optimizer 
    optimizer = tf.train.AdamOptimizer(lr)
    final_loss = tf.contrib.bayesflow.stochastic_graph.surrogate_loss(loss)
    gen_vars = [v for v in tf.trainable_variables() if v.op.name.startswith('gen')]
    min_op = optimizer.minimize(final_loss, var_list = gen_vars)
    
    # Group operations.
    train_op = tf.group(min_op, maintain_avg_op)
    return final_loss, train_op


def train_discriminator(sequences, labels):
    '''Train Discriminator via cross entropy'''
    loss = []
    for seq, lab in zip(sequences, labels):
        loss.append(tf.contrib.losses.log_loss(seq, lab))
    
    final_loss = tf.reduce_sum(loss) / batch_size / seq_length
    dis_vars = [v for v in tf.trainable_variables() if v.op.name.startswith('dis')]
    
    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)
    train_op = optimizer.minimize(final_loss, var_list = dis_vars)
    
    
    

### Check Loss Calculation

In [24]:
logits = tf.constant([[0.5]], dtype=tf.float32)
labels = tf.constant([[0]])

loss = tf.contrib.losses.log_loss(logits, labels)
with tf.Session() as sess:
    print sess.run(loss)


0.693147


## Generative Adversarial Networks Training.

In [42]:
tf.reset_default_graph()

fake_seq = generator()
print len(fake_seq)
preds = discriminator(fake_seq)

loss, train_op = train_generator(preds)

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    for t in xrange(num_trials):
        sess.run(train_op)
        if t % print_every == 0:
            print sess.run(loss) 

4
[[ 2.22107077  1.57640052]]
[[ 0.31343961 -0.01342641]]
[[ 0.21456057  0.16737241]]
[[ 0.21627289 -0.32114762]]
[[ 0.25093225 -0.06628224]]
[[ 0.34141546  0.17728008]]
[[ 0.33368659 -0.06670886]]
[[ 0.30806732  0.07261313]]
[[ 0.34760535 -0.22436023]]
[[-0.01210876 -0.97494161]]


# TODO
* Generalize for random input $z_{gen}$